https://medium.com/@tommynineteenies/data-scraping-with-scrapy-and-excel-57d04605d802

In [16]:
import scrapy
from scrapy.crawler import CrawlerProcess
import win32com.client as win32
import os





In [17]:
class GPUSpider(scrapy.Spider):
    name = "GPU"
  
    def start_requests(self):
        # go to the best buy website
        url = "https://www.bestbuy.com/site/searchpage.jsp?st=gpu"
        
        # Set the headers here because best buy disable random requests, we sending the request acting as Firefox
        headers = {'User-agent': 'Opera/94.0.0.0 ',"Accept-Encoding": "*",    # Mozilla/5.0
        "Connection": "keep-alive"}

        # Sending the request
        yield scrapy.http.Request(url, headers=headers)

    def parse(self, response):
        # get html element has the class of sku_item
        for gpu in response.css('li.sku-item'):
            yield {
                'product_title': gpu.css('div.sku-title a::text').get(), # select div tag with css name of sku-title
                'product_price': gpu.css('div.priceView-customer-price span::text').get(), # select div tag with css name of customer-price
                'availability': gpu.xpath("//button[contains(@class, 'add-to-cart-button')]/@data-button-state").get() # select data-button-state attibute of add-to-cart-button
            }
        # click on the next page button
        next_page = response.css('a.sku-list-page-next').attrib['href']
        # This is a recursive function to parse next page until there is no next page
        if next_page is not None:
            headers = {'User-agent': 'Mozilla/5.0',"Accept-Encoding": "*",
            "Connection": "keep-alive"}
            yield response.follow(next_page, callback=self.parse, headers = headers)





In [18]:
if __name__ == "__main__":
    print("Program started... ")
    # set the output file as csv
    setting = {
        "FEEDS": {
            "gpu.csv": {"format": "csv"},
        }
    }
    # start the crawler 
    process = CrawlerProcess(settings = setting)
    process.crawl(GPUSpider)
    process.start()
    # Use excel to open gpu.csv (the file we just output it )
    wb_path = "gpu.csv"
    if os.path.exists(wb_path):
        excel = win32.gencache.EnsureDispatch('Excel.Application')
        excel.Workbooks.Open(os.path.abspath(wb_path), ReadOnly=1)
        excel.Visible = 1
        excel.WindowState = win32.constants.xlMaximized
    print("Program finished")

2023-02-03 00:04:55 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2023-02-03 00:04:55 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1o  3 May 2022), cryptography 37.0.1, Platform Windows-10-10.0.19044-SP0
2023-02-03 00:04:55 [scrapy.crawler] INFO: Overridden settings:
{}


Program started... 


ReactorAlreadyInstalledError: reactor already installed

In [21]:
# select div tag with css name of sku-title
product_title: gpu.css('div.sku-title a::text').get()
# select div tag with css name of customer-price
product_price: gpu.css('div.priceView-customer-price span::text').get()
# select data-button-state attibute of add-to-cart-button
availability: gpu.xpath("//button[contains(@class, 'add-to-cart-button')]/@data-button-state").get()

AttributeError: 'str' object has no attribute 'css'